In [1]:
# Convolutional neural network from scratch 
import cv2
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('mnist_train.csv')
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = np.array(data) # transform our csv into array 
lines, cols = data.shape # lines = 42000 / columns = 785 bcz the images are 28x28=784 pixels +1 with the beggining cols of label
np.random.shuffle(data) # because the train data are possibly sorted with a sequences of 0 then 1 then 2 and so on 
# we shuffe the dataset in order to avoid overfitting
data_validation = data[0:1000].T # Data transposition is performed in this particular context to facilitate 
# the manipulation of examples and features. Some machine learning algorithms and libraries prefer 
# this matrix representation with examples in columns rather than rows.
# data validation variable is used to evaluate and confirm the performance of our future model while training
label_val = data_validation[0]
result_val = data_validation[1:cols]

data_train = data[1000:lines].T
label_train = data_train[0] # we did the transpose so le first line is compose only by label
result_train = data_train[1:cols]

In [7]:
def init_params() :
    # W => weights / b => bias ... In the beginning both are generated randomly 
    # 1st param : nb neurons on the actual layer / 2st param : nb neurons of the previous layer
    neuronsLayerOne = 10 
    neuronsLayerTwo = 10
    W1 = np.random.rand(neuronsLayerOne,cols-1) # -1 => to understand refer to what we said on the line of creation of cols
    b1 = np.random.rand(neuronsLayerOne,1) # which means for each neurons we generate a random number which will corresponding to the bias
    # bias doesn't have to be necessary between 0 and 1 ! 
    W2 = np.random.rand(neuronsLayerTwo,neuronsLayerOne)
    b2 = np.random.rand(neuronsLayerTwo,1)
    return W1, b1, W2, b2
def ReLu(Z) : #activation function non linear 
    return np.maximum(0,Z)
def softmax(Z) : 
    return np.exp(Z)/np.sum(np.exp(Z))
def forward_propagation(W1, b1, W2, b2, X) : # X is the entry (pixels 1 image)
    # forward propagation is when you take images and run them through the CNN / It's a linear transformation
    Z1 = W1.dot(X)+b1 # X -> several lines and 1 column due to the transpose ! (l,c) x (l,c) W1 -> single value of the neurons
    A1 = ReLu(Z1)
    Z2 = W2.dot(X)+b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

array([[9.14776890e-01, 5.52458929e-01, 4.40784112e-01, ...,
        7.31945062e-01, 1.74036607e-01, 3.92519962e-01],
       [1.82781717e-02, 5.50615057e-01, 1.51433558e-01, ...,
        3.25995992e-04, 3.02063412e-01, 2.71691368e-01],
       [8.22788607e-01, 6.46954805e-01, 8.88385360e-01, ...,
        4.46758068e-01, 1.46098673e-01, 6.58316060e-01],
       ...,
       [1.08283121e-01, 2.51006234e-02, 1.39442068e-01, ...,
        8.43862151e-01, 2.87579537e-02, 4.78108525e-01],
       [3.12560343e-01, 7.35633926e-01, 9.76937446e-01, ...,
        7.65421141e-01, 4.62650573e-01, 6.70648911e-02],
       [1.77378221e-01, 5.23974791e-01, 7.29880250e-01, ...,
        9.95491771e-01, 9.36211524e-02, 4.90057402e-01]])